In [6]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

In [73]:
discriminator = keras.Sequential([
    keras.Input(shape=(32,32,3)),
    
    layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'),
    layers.LeakyReLU(alpha=0.2),
    
    layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'),
    layers.LeakyReLU(alpha=0.2),
    
    layers.GlobalMaxPooling2D(),
    layers.Dense(1)
], name='D')

In [74]:
latent_dim = 128

generator = keras.Sequential([
    keras.Input(shape=(latent_dim,)),
    
    layers.Dense(7*7*128),
    layers.LeakyReLU(alpha=0.2),
    layers.Reshape((7, 7, 128)),
    layers.Conv2DTranspose(128, (4, 4), strides=2, padding='same'),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2DTranspose(128, (4, 4), strides=2, padding='same'),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2D(1, (7, 7), padding='same', activation='sigmoid')
], name='G')

In [10]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        
    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    @tf.function    
    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        
        generated_images = self.generator(random_latent_vectors)
        
        combined_images = tf.concat([generated_images, real_images], axis=0)
        
        labels = tf.concat([tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0)
        
        labels += 0.04 * tf.random.uniform(tf.shape(labels))
        
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
            
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(zip(grads, 
                                             self.discriminator.trainable_weights))

        random_latent_vectors = tf.random.normal(shape=(batch_size, 1))
        misleading_labels = tf.zeros((batch_size, 1))
        
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
            
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads,
                                             self.generator.trainable_weights))

        return {'d_loss': d_loss, 'g_loss': g_loss}
        
        

In [76]:
batch_size = 64
n_epochs = 4

In [65]:
(x_train, _), (x_test, _) = keras.datasets.cifar10.load_data()

In [66]:
all_digits = np.concatenate([x_train, x_test])
all_digits = all_digits.astype('float32') / 255.0
all_digits = np.reshape(all_digits, (-1, 32, 32, 3))

In [67]:
dataset = tf.data.Dataset.from_tensor_slices(all_digits)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

In [69]:
x_test.shape

(10000, 32, 32, 3)

In [70]:
d_optimizer = keras.optimizers.Adam(learning_rate=1e-3)
g_optimizer = keras.optimizers.Adam(learning_rate=1e-3)

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [71]:
model = GAN(discriminator, generator, latent_dim)
model.compile(d_optimizer, g_optimizer, loss_fn)

In [75]:
for ep in range(n_epochs):

    for idx, real_images in enumerate(dataset):
        d_loss, g_loss, generated_images = model.train_step(real_images)
        
        if idx % 200 == 0:
            print(f"epoch #{ep}, step #{idx}, D-loss={d_loss}")
            print(f"epoch #{ep}, step #{idx}, G-loss={g_loss}")
        
        
            # Save one generated image
            img = tf.keras.preprocessing.image.array_to_img(
                generated_images[0] * 255.0, scale=False
            )
            img.save(os.path.join(save_dir, "generated_img" + str(idx) + ".png"))


InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [64,28,28,1] vs. shape[1] = [64,32,32,3] [Op:ConcatV2] name: concat

In [38]:
encoder_voc = 1_000
decoder_voc = 2_000

encoder_input = layers.Input(shape=(None, ))
encoder_embedded = layers.Embedding(input_dim=encoder_voc, output_dim=64)(encoder_input)

output, state_h, state_c = layers.LSTM(units=64, 
                                       return_state=True, 
                                       name='encoder')(encoder_embedded)

encoder_state = [state_h, state_c]

decoder_input = layers.Input(shape=(None, ))
decoder_embedded = layers.Embedding(input_dim=decoder_voc, output_dim=64)(decoder_input)

decoder_output = layers.LSTM(units=64, 
                                       name='decoder')(decoder_embedded,
                                                       initial_state=encoder_state)

output = layers.Dense(10)(decoder_output)

In [41]:
model = keras.Model([encoder_input, decoder_input], output)

In [42]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 64)     64000       input_10[0][0]                   
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 64)     128000      input_11[0][0]                   
_______________________________________________________________________________________

In [44]:
model = keras.Sequential()
model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True), 
                               input_shape=(5,10)))

model.add(layers.Bidirectional(layers.LSTM(32)))

model.add(layers.Dense(10))

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 5, 128)            38400     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 80,266
Trainable params: 80,266
Non-trainable params: 0
_________________________________________________________________


In [50]:
# создание базовой модели - фундамент
base_model = keras.applications.EfficientNetB0(weights='imagenet',
                                               input_shape=(32, 32, 3),
                                               include_top=False)

16711680/16705208 [==============================] - 253s 15us/step


In [51]:
# load CIFAR-10 dataset
# cifar10

In [52]:
# заморозить базовую модель
base_model.trainable = False

In [55]:
# создание новой модели на верху
inputs = keras.Input(shape=(32, 32, 3))
# подсоединение входного слоя к базовой модели
x = base_model(inputs, training=False)
# 
x = keras.layers.GlobalAveragePooling2D()(x)
# на конце плотный слой 
outputs = keras.layers.Dense(1)(x)

model = keras.Model(inputs, outputs)

In [58]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss=keras.losses.categorical_crossentropy(),
              metrics=[keras.metrics.Accuracy()])

TypeError: categorical_crossentropy() missing 2 required positional arguments: 'y_true' and 'y_pred'